In [34]:
#Importando Bibliotecas Pandas e Numpy
import pandas as pd
import numpy as np
import os, sys
import time

#Importando BiBliotecas DASK  (alternativa ao Pandas)
import dask.dataframe as dd

#Otimizador Giovanni
from sklearnex import patch_sklearn 
patch_sklearn()


CSV_PATH = "/home/gi/Desktop/Semestre8/Inteligencia_Computacional/TP1_csvs" # Path Giovanni
# CSV_PATH = "/home/gi/Desktop/Semestre8/Inteligencia_Computacional" # Path Giovanni
# CSV_PATH = "E:/Documentos/CEFET/OneDrive/Documentos/2ECOM067_INTELIGENCIA-COMPUTACIONAL-I_T01/TP1/bases/" # Path Jorge
start_time = time.time()

import warnings
warnings.simplefilter("ignore")
#Carregando Base de dados principal
df_test  = pd.read_csv(f"{CSV_PATH}/df_test.csv")
df_train = pd.read_csv(f"{CSV_PATH}/df_train_full.csv")


Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [31]:
df_train = df_train.sample(frac=0.6, random_state=35) # Amostrando 10% da base de treino
df_train.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
Index: 92254 entries, 23711 to 99693
Data columns (total 195 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    TARGET                        92254 non-null  int64  
 1    CODE_GENDER                   92254 non-null  object 
 2    FLAG_OWN_CAR                  92254 non-null  object 
 3    FLAG_OWN_REALTY               92254 non-null  object 
 4    CNT_CHILDREN                  92254 non-null  int64  
 5    AMT_INCOME_TOTAL              92254 non-null  float64
 6    NAME_INCOME_TYPE              92254 non-null  object 
 7    NAME_EDUCATION_TYPE           92254 non-null  object 
 8    NAME_FAMILY_STATUS            92254 non-null  object 
 9    NAME_HOUSING_TYPE             92254 non-null  object 
 10   REGION_POPULATION_RELATIVE    92254 non-null  float64
 11   DAYS_BIRTH                    92254 non-null  int64  
 12   DAYS_EMPLOYED                 92254 non-null 

In [ ]:
# Treinamento
# Importanto as Bibliotecas dos classificadores
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

atributos_categoricos = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
    'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
    'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE',
    'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE', 'STATUS',
    'NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
    'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
    'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
    'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
    'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY',
    'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'NAME_CONTRACT_STATUS'
]
colunas_para_dropar = [
    'CODE_GENDER_XNA', 'CREDIT_CURRENCY_currency4',
    'CREDIT_TYPE_Loanforthepurchaseofequipment', 'CREDIT_TYPE_Mobileoperatorloan',
    'CREDIT_TYPE_Realestateloan', 'NAME_CONTRACT_STATUS_Refused',
    'NAME_GOODS_CATEGORY_Insurance', 'NAME_INCOME_TYPE_Maternityleave'
]

import re
X_train_local = pd.get_dummies(df_train, columns=atributos_categoricos, drop_first=True)
X_train_local.columns = [re.sub(r'[^\w]', '', col) for col in X_train_local.columns]
y_train_local = X_train_local['TARGET']
X_train_local = X_train_local.drop(columns=['TARGET'])
X_train_local = X_train_local.drop(columns=colunas_para_dropar)

In [37]:
# Treinamento
# Importanto as Bibliotecas dos classificadores
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

atributos_categoricos = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
    'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
    'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE',
    'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE', 'STATUS',
    'NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
    'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
    'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
    'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
    'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY',
    'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'NAME_CONTRACT_STATUS'
]
colunas_para_dropar = [
    'CODE_GENDER_XNA', 'CREDIT_CURRENCY_currency4',
    'CREDIT_TYPE_Loanforthepurchaseofequipment', 'CREDIT_TYPE_Mobileoperatorloan',
    'CREDIT_TYPE_Realestateloan', 'NAME_CONTRACT_STATUS_Refused',
    'NAME_GOODS_CATEGORY_Insurance', 'NAME_INCOME_TYPE_Maternityleave'
]


import re
X_train_local = pd.get_dummies(df_train, columns=atributos_categoricos, drop_first=True)
X_train_local.columns = [re.sub(r'[^\w]', '', col) for col in X_train_local.columns]
X_train_local = X_train_local.sample(frac=0.16, random_state=35) # Amostrando 10% da base de treino
y_train_local = X_train_local['TARGET']
X_train_local = X_train_local.drop(columns=['TARGET'])
X_train_local = X_train_local.drop(columns=colunas_para_dropar)


In [5]:
def tem_caractere_proibido(col):
    return bool(re.search(r'[^a-zA-Z0-9_]', col))

colunas_invalidas = [col for col in X_train_local if tem_caractere_proibido(col)]
print("Colunas com caracteres inválidos:", colunas_invalidas)

Colunas com caracteres inválidos: []


In [11]:
# Definicao de hiperparametros para o GridSearch
from sklearn.model_selection import GridSearchCV

def melhores_parametros_modelo(param_grid, modelo_base, X_train, y_train):
    """
    Funcao para encontrar os melhores hiperparametros para o modelo
    """
    grid = GridSearchCV(estimator=modelo_base, param_grid=param_grid,
                    scoring='roc_auc', cv=10, n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    print(f"Melhores hiperparametros: {grid.best_params_}, ROC AUC: {grid.best_score_}")

In [ ]:
# Definicao de hiperparametros para o GridSearch
from sklearn.model_selection import GridSearchCV

# Define os parâmetros a serem testados para cada modelo
param_grid_xgb = {
    'max_depth': [4, 5, 6],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 150, 200],
    'subsample': [0.8, 1.0]
}

param_grid_lgbm = {
    'max_depth': [4, 6],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 150, 200],
    'subsample': [0.8, 1.0]
}

param_grid_catboost = {
    'depth': [4, 6],
    'learning_rate': [0.1, 0.2, 0.3],
    'iterations': [100, 150, 200],
    'l2_leaf_reg': [1, 3]
}

# Cria o classificador base
modelo_xgb = XGBClassifier(random_state=42)
modelo_lgbm = LGBMClassifier(random_state=42, force_col_wise=True)
modelo_catboost = CatBoostClassifier(random_state=42, verbose=0)

In [ ]:
melhores_parametros_modelo(param_grid_xgb, modelo_xgb, X_train_local, y_train_local)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Melhores hiperparametros: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}, ROC AUC: 0.7291956437034702


In [16]:
# Aplica o GridSearch nos modelos
# melhores_parametros_modelo(param_grid_xgb, modelo_xgb, X_train_local, y_train_local)
melhores_parametros_modelo(param_grid_lgbm, modelo_lgbm, X_train_local, y_train_local)
melhores_parametros_modelo(param_grid_catboost, modelo_catboost, X_train_local, y_train_local)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[LightGBM] [Info] Number of positive: 2032, number of negative: 22568
[LightGBM] [Info] Total Bins 20057
[LightGBM] [Info] Number of data points in the train set: 24600, number of used features: 359
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 2032, number of negative: 22568
[LightGBM] [Info] Number of positive: 2033, number of negative: 22568
[LightGBM] [Info] Number of positive: 2033, number of negative: 22568
[LightGBM] [Info] Total Bins 20057
[LightGBM] [Info] Number of data points in the train set: 24600, number of used features: 359
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Info] Number of positive: 2033, number of neg

In [38]:
# Combinacao com Voting Classifier
# Importando Bibliotecas pertinentes
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Fazendo validação cruzada (manter a proporção de classes)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [39]:
def avaliar_modelo_teste(modelos, X, y, nome="Modelo único de votação"): # Modelo único
    roc_aucs = []
    for train_idx, test_idx in skf.split(X, y):
        soma_preds = np.zeros(len(test_idx))

        for modelo in modelos:
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            modelo.fit(X_train, y_train)
            preds = modelo.predict(X_test)

            soma_preds += preds  # soma as previsões

        # Votação: se 3 ou mais classificadores disseram 1 → classe 1
        preds_final = [1 if s > 2 else 0 for s in soma_preds]

        # Calcula AUC para esse fold
        roc_aucs.append(roc_auc_score(y_test, preds_final))

    print(f"{nome}: ROC AUC = {np.mean(roc_aucs):.4f} ± {np.std(roc_aucs):.4f}")

In [6]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Define os parâmetros a serem testados

xgboost_params = {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}
lgbm_params = {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}
catboost_params = {'depth': 4, 'iterations': 150, 'l2_leaf_reg': 3, 'learning_rate': 0.1}

# Cria o classificador base
xgb = XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100,subsample=0.8,random_state=42, use_label_encoder=False, eval_metric='logloss')
lgbm = LGBMClassifier(learning_rate=0.1,max_depth=4,n_estimators=100,subsample=0.8,random_state=42, force_col_wise=True)
catboost = CatBoostClassifier(depth=4, iterations=150, l2_leaf_reg=3, learning_rate=0.1, random_state=42, verbose=0)

avaliar_modelo_teste([xgb, lgbm, catboost], X_train_local, y_train_local, "Modelo único de votação")

[LightGBM] [Info] Number of positive: 14530, number of negative: 165363
[LightGBM] [Info] Total Bins 21486
[LightGBM] [Info] Number of data points in the train set: 179893, number of used features: 398
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080770 -> initscore=-2.431928
[LightGBM] [Info] Start training from score -2.431928
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [16]:
def avaliar_modelo_individual(modelo, X_train_local, y_train_local, nome="Modelo único de votação"):
    """
    Funcao para avaliar o modelo individualmente
    """
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_aucs = []
    for train_idx, test_idx in skf.split(X_train_local, y_train_local):
        X_train, X_test = X_train_local.iloc[train_idx], X_train_local.iloc[test_idx]
        y_train, y_test = y_train_local.iloc[train_idx], y_train_local.iloc[test_idx]

        modelo.fit(X_train, y_train)
        preds = modelo.predict(X_test)

        # Calcula AUC para esse fold
        roc_aucs.append(roc_auc_score(y_test, preds))

    print(f"{nome}: ROC AUC = {np.mean(roc_aucs):.4f} ± {np.std(roc_aucs):.4f}")

In [18]:
# Teste com voting classifier
voting_clf = VotingClassifier(estimators=[
    ('xgb', xgb),
    ('lgbm', lgbm),
    ('catboost', catboost)
], voting='hard')
avaliar_modelo_individual(voting_clf, X_train_local, y_train_local, "Voting Classifier")

[LightGBM] [Info] Number of positive: 2286, number of negative: 25389
[LightGBM] [Info] Total Bins 20166
[LightGBM] [Info] Number of data points in the train set: 27675, number of used features: 362
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [ ]:
# Testando com Stacking classifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stacking = StackingClassifier(
    estimators=[
        ('xgb', xgb),
        ('lgbm', lgbm),
        ('catboost', catboost)
    ],
    final_estimator=LogisticRegression(),
    cv=10
)
avaliar_modelo_individual(stacking, X_train_local, y_train_local, "Stacking Classifier")

NameError: name 'xgb' is not defined

In [ ]:
# Treinando so com a tabela principal
X_train_alternativo = pd.read_csv(f"{CSV_PATH}/application_train.csv")
X_train_alternativo = X_train_alternativo.sample(frac=0.6, random_state=35) # Amostrando 10% da base de treino
colunas_existentes = [col for col in atributos_categoricos if col in X_train_alternativo.columns]
X_train_alternativo = pd.get_dummies(X_train_alternativo, columns=colunas_existentes, drop_first=True)
X_train_alternativo.columns = [re.sub(r'[^\w]', '', col) for col in X_train_alternativo.columns]
y_train_alternativo = X_train_alternativo['TARGET']
X_train_alternativo = X_train_alternativo.drop(columns=("TARGET"))
xgb_alternativo = XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100,subsample=0.8,random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_alternativo.fit(X_train_alternativo, y_train_alternativo)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [44]:
# Submissao para o kaggle
# Testando com Stacking classifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
# Treinando o modelo com a base completa
stacking = StackingClassifier(
    estimators=[
        ('xgb', xgb),
        ('lgbm', lgbm),
        ('catboost', catboost)
    ],
    final_estimator=LogisticRegression(),
    cv=10
)

# Preparando os dados para o treinamento
"""
X_train_full = pd.read_csv(f"{CSV_PATH}/df_train_full.csv")
y_train_full = X_train_full['TARGET']
X_train_full = X_train_full.drop(columns=['TARGET'])
X_train_full = pd.get_dummies(X_train_full, columns=atributos_categoricos, drop_first=True)
X_train_full.columns = [re.sub(r'[^\w]', '', col) for col in X_train_full.columns]
 """
 
X_train_full = X_train_local
y_train_full = y_train_local

col_sk_id_completo = set(X_train_full['SK_ID_CURR'])


skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for train_idx, test_idx in skf.split(X_train_full, y_train_full):
    X_train, X_test = X_train_full.iloc[train_idx], X_train_full.iloc[test_idx]
    y_train, y_test = y_train_local.iloc[train_idx], y_train_local.iloc[test_idx]

    stacking.fit(X_train, y_train)
    preds = stacking.predict(X_test)

    # Calcula AUC para esse fold
    print(f"ROC AUC = {roc_auc_score(y_test, preds):.4f}")
    
# Treinando so com a tabela principal

[LightGBM] [Info] Number of positive: 3627, number of negative: 40654
[LightGBM] [Info] Total Bins 20515
[LightGBM] [Info] Number of data points in the train set: 44281, number of used features: 372
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081909 -> initscore=-2.416691
[LightGBM] [Info] Start training from score -2.416691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [47]:
# Fazendo previsões na base de teste

X_test = pd.read_csv(f"{CSV_PATH}/df_test.csv")
X_test = pd.get_dummies(X_test, columns=atributos_categoricos, drop_first=True)
X_test.columns = [re.sub(r'[^\w]', '', col) for col in X_test.columns]
print(f"quantidade de colunas X_test {X_test.shape}, quantidade de colunas X_train {X_train_local.shape}")



if len(X_test.columns) !=len( X_train_local.columns):
    print("Colunas diferentes entre X_train e X_test")
    
colunas_faltando_em_x_train = set(X_test.columns) - set(X_train_local.columns)

# Removendo colunas que não estão no X_train
X_test = X_test.drop(columns=colunas_faltando_em_x_train)
    

# Faz o merge apenas dos SK_ID_CURR presentes nos dois DataFrames
conjunto_treino_total = X_test.merge(
    X_train_local[['SK_ID_CURR']], on='SK_ID_CURR', how='inner'
)

# Agora pega os que NÃO estão no conjunto anterior
conjunto_treino_restante = X_test[~X_test['SK_ID_CURR'].isin(conjunto_treino_total['SK_ID_CURR'])]



# Predicao e salvando csv
p1 = stacking.predict(conjunto_treino_total)
df_sub1 = pd.DataFrame({"SK_ID_CURR": conjunto_treino_total['SK_ID_CURR'], 'TARGET': p1})
p2 = stacking.predict(conjunto_treino_restante)
df_sub2 = pd.DataFrame({"SK_ID_CURR": conjunto_treino_restante['SK_ID_CURR'], 'TARGET': p2})
# Concatenando os dois dataframes
df_submission = pd.concat([df_sub1, df_sub2], axis=0)
df_submission.to_csv('tp1_kagglexgb.csv', index=False)

quantidade de colunas X_test (48744, 413), quantidade de colunas X_train (49202, 415)
Colunas diferentes entre X_train e X_test


ValueError: feature_names mismatch: ['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_